In [1]:
#Import Dependencies
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
from datetime import datetime as dt

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import table, column
import datetime

In [3]:
#Read in Data from CSV
drug_data_csv = "Resources/drug_deaths.csv"
income_data_csv = "Resources/per_capita_personal_income.csv"
drug_data = pd.read_csv(drug_data_csv)
income_data = pd.read_csv(income_data_csv)
drug_data.head()
#income_data.head()

,Unnamed: 0,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,...,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
0,0,14-0273,06/28/2014 12:00:00 AM,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,NaN,0,0,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
1,1,13-0102,03/21/2013 12:00:00 AM,0.0,48.0,Male,Black,NORWALK,NaN,NaN,...,0,0,0,NaN,0,0,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
2,2,16-0165,03/13/2016 12:00:00 AM,0.0,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,...,0,0,0,NaN,0,1,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
3,3,16-0208,03/31/2016 12:00:00 AM,0.0,23.0,Male,White,RYE,WESTCHESTER,NY,...,0,0,0,NaN,0,1,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
4,4,13-0052,02/13/2013 12:00:00 AM,0.0,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,...,0,0,0,NaN,0,0,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


In [41]:
#Desired Data
new_drug_data = drug_data[['ID','Date', 'Age', 'ResidenceState', 'COD', 'Heroin', 'Cocaine', 'Fentanyl', 'Oxycodone','Oxymorphone', 'Ethanol','Hydrocodone', 'Benzodiazepine', 'Methadone','Amphet', 'Tramad']].copy()
#new_drug_data.head()

new_income_data = income_data.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11 ,12 ,13, 14, 15, 16,17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,28,29]]
new_income_data.head()

,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,2014:Q2,...,2016:Q3,2016:Q4,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,43971,44381,44191,45616,44353,44720,44910,45171,45909,46616,...,49734,50134,50794,51286,51773,52428,53022,53532,54122,54578
1,35571,35873,35507,36323,35900,36009,36176,35959,36269,36846,...,39004,39243,39823,40112,40305,40731,41148,41409,41675,42162
2,53535,53819,53460,54042,52634,52545,53053,53253,54867,55703,...,56218,56553,56564,56869,57597,58164,58846,59239,60258,60717
3,36104,36470,36217,37364,36701,37090,37254,37427,38134,38522,...,41720,41857,42608,42966,43597,43955,44688,44994,45551,45890
4,35988,36556,36187,37595,36500,36639,36803,36739,37782,38584,...,40915,41214,41757,41871,42027,42567,42986,43338,43849,44447


Data Cleaning: Drug Overdose Data

In [16]:
#Extract only year from Date Column
new_drug_data['Date'] = pd.to_datetime(new_drug_data['Date'])
new_drug_data['Date'] = pd.DatetimeIndex(new_drug_data['Date']).year

In [17]:
#Extract only year from Date Column
new_drug_data['Date'] = new_drug_data['Date'].fillna(0)
new_drug_data['Date'] = new_drug_data['Date'].astype(int)
new_drug_data = new_drug_data.set_index('Date')
new_drug_data.head()

,ID,Age,ResidenceState,COD,Heroin,Cocaine,Fentanyl,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad
Date,,,,,,,,,,,,,,,
2014,14-0273,NaN,NaN,"Acute fent, hydrocod, benzodiazepine",0,0,1,0,0,0,1,1,0,0,0
2013,13-0102,48.0,NaN,Cocaine Intoxication,0,1,0,0,0,0,0,0,0,0,0
2016,16-0165,30.0,CT,Acute Heroin and Cocaine Intoxication,1,1,0,0,0,0,0,0,0,0,0
2016,16-0208,23.0,NY,Acute Fentanyl and Morphine Intoxication,1,0,1,0,0,0,0,0,0,0,0
2013,13-0052,22.0,NaN,Fentanyl Intoxication,0,0,1,0,0,0,0,0,0,0,0


In [18]:
#Edit columns not int
new_drug_data["Fentanyl"].unique()
new_drug_data["Fentanyl"] = new_drug_data.apply(lambda row: int(row["Fentanyl"][:1]), axis=1)

In [25]:
#Combine data to be per year (2012 to 2018)
drug_overdose_temp = new_drug_data.groupby(by='Date').agg({'Fentanyl': 'sum',
                            'Heroin': 'sum',
                            'Cocaine': 'sum',
                            'Oxycodone': 'sum',
                            'Oxymorphone': 'sum',
                            'Ethanol' : 'sum',
                            'Hydrocodone' : 'sum',
                            'Benzodiazepine': 'sum',
                            'Methadone': 'sum',
                            'Amphet' : 'sum',
                            'Tramad' : 'sum'}).reset_index()


In [26]:
drug_overdose = drug_overdose_temp.drop(drug_overdose_temp.index[0])
drug_overdose

,Date,Fentanyl,Heroin,Cocaine,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad
1,2012,14,174,105,70,30,61,15,49,33,7,8
2,2013,37,257,147,74,16,83,19,78,47,4,7
3,2014,75,325,127,101,29,126,15,157,51,13,15
4,2015,188,416,174,95,6,176,20,221,72,20,16
5,2016,482,494,275,110,7,255,20,242,84,19,17
6,2017,676,474,347,95,8,290,15,330,99,40,28
7,2018,760,389,345,62,12,256,14,266,88,56,39


Data Cleaning: Personal Income Per Capita

In [34]:
#Rename Columns Date
new_income_data.columns.values[2:6] =["2012"]
new_income_data.columns.values[6:10] =["2013"]
new_income_data.columns.values[10:14] =["2014"]
new_income_data.columns.values[14:18] =["2015"]
new_income_data.columns.values[18:22] =["2016"]
new_income_data.columns.values[22:26] =["2017"]
new_income_data.columns.values[26:30] =["2018"]

In [42]:
new_income_data

,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,2014:Q2,...,2016:Q3,2016:Q4,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,43971,44381,44191,45616,44353,44720,44910,45171,45909,46616,...,49734,50134,50794,51286,51773,52428,53022,53532,54122,54578
1,35571,35873,35507,36323,35900,36009,36176,35959,36269,36846,...,39004,39243,39823,40112,40305,40731,41148,41409,41675,42162
2,53535,53819,53460,54042,52634,52545,53053,53253,54867,55703,...,56218,56553,56564,56869,57597,58164,58846,59239,60258,60717
3,36104,36470,36217,37364,36701,37090,37254,37427,38134,38522,...,41720,41857,42608,42966,43597,43955,44688,44994,45551,45890
4,35988,36556,36187,37595,36500,36639,36803,36739,37782,38584,...,40915,41214,41757,41871,42027,42567,42986,43338,43849,44447
5,47180,47675,47567,50008,47595,48314,48707,49335,49948,50807,...,56808,57297,57841,58456,59085,59890,60344,61195,61992,62543
6,44951,45360,45076,47067,46530,47185,47656,48191,49665,50524,...,52535,53113,54091,54807,55582,56626,57316,58040,58942,59646
7,63486,64142,63383,65474,62284,62500,62717,63057,64101,65085,...,68668,68707,69150,69996,70372,71051,72123,72718,73919,73984
8,43558,44017,43685,44961,43934,44414,44454,44767,45293,45805,...,48948,49344,49926,50056,50809,51197,51995,52503,53223,53573
9,67945,68239,67301,69029,67577,67834,67687,68161,70375,71148,...,78252,78898,79152,79541,80316,80957,81470,82040,83224,83455


In [38]:
#new_income_data.T

In [48]:
#data1 = new_income_data.loc[]
new_income_data['Income 2012'] = pd.concat([new_income_data['2012:Q1'], new_income_data['2012:Q2'],new_income_data['2012:Q3'],new_income_data['2012:Q4']],                  
                      ignore_index = True,
                      sort = False)

new_income_data['Income 2013'] = pd.concat([new_income_data['2013:Q1'], new_income_data['2013:Q2'],new_income_data['2013:Q3'],new_income_data['2013:Q4']],                  
                      ignore_index = True,
                      sort = False)

C:\Users\COOPERKK\AppData\Local\Temp\ipykernel_21324\4033399534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_income_data['Income 2012'] = pd.concat([new_income_data['2012:Q1'], new_income_data['2012:Q2'],new_income_data['2012:Q3'],new_income_data['2012:Q4']],


In [49]:
new_income_data.head()

,2012:Q1,2012:Q2,2012:Q3,2012:Q4,2013:Q1,2013:Q2,2013:Q3,2013:Q4,2014:Q1,2014:Q2,...,2017:Q1,2017:Q2,2017:Q3,2017:Q4,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Income 2012,Income 2013
0,43971,44381,44191,45616,44353,44720,44910,45171,45909,46616,...,50794,51286,51773,52428,53022,53532,54122,54578,43971,44353
1,35571,35873,35507,36323,35900,36009,36176,35959,36269,36846,...,39823,40112,40305,40731,41148,41409,41675,42162,35571,35900
2,53535,53819,53460,54042,52634,52545,53053,53253,54867,55703,...,56564,56869,57597,58164,58846,59239,60258,60717,53535,52634
3,36104,36470,36217,37364,36701,37090,37254,37427,38134,38522,...,42608,42966,43597,43955,44688,44994,45551,45890,36104,36701
4,35988,36556,36187,37595,36500,36639,36803,36739,37782,38584,...,41757,41871,42027,42567,42986,43338,43849,44447,35988,36500


In [ ]:
average_df = new_income_data.mean

In [ ]:
#Convert State Abbrev.
state_2 = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

state_2 = {state: abbrev for state, abbrev in state_2.items()}

In [ ]:
#New State Col. (abbrev so it will match drug_overdose states)
new_income_data['abbrev'] = new_income_data['GeoName'].map(state_2)
new_income_data = new_income_data.drop(columns=['GeoName'])
new_income_data = new_income_data.rename(columns={'abbrev':"State"})
new_income_data = new_income_data.dropna() 
new_income_data

Connect to Local Database 

In [ ]:
#Connect to DB
protocol = 'postgresql'
username = 'postgres'
password = ''
host = 'localhost'
port = 5433
database_name = 'drug_overdose_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
#engine.table_names()